Generates the following LUT:

LUT[N-bit, unsigned, fully integer binary number] = N-bit, unsigned, fully *fractional* binary number

In [1]:
import os
import itertools
import numpy as np
import copy
from tqdm import tqdm

# global variables
N_BITS_IN = 8
N_BITS_OUT = 24

MAX_VAL_IN_BIN = "1"*N_BITS_IN
MAX_VAL_IN_INT = 0
for i in range(N_BITS_IN):
    MAX_VAL_IN_INT += 2**i
print(f"MAX_VAL_IN_INT: {MAX_VAL_IN_INT}")

MAX_VAL_OUT_BIN = "1"*N_BITS_OUT
MAX_VAL_OUT_FRAC = 0
for i in range(N_BITS_OUT):
    MAX_VAL_OUT_FRAC += 2**(-i-1)
print(f"MAX_VAL_OUT_FLOAT: {MAX_VAL_OUT_FRAC}")
MIN_VAL_OUT_FRAC = 2**(-N_BITS_OUT-1)
print(f"MIN_VAL_OUT_FRAC: {MIN_VAL_OUT_FRAC}")

np.random.seed(0)

MAX_VAL_IN_INT: 255
MAX_VAL_OUT_FLOAT: 0.9999999403953552
MIN_VAL_OUT_FRAC: 2.9802322387695312e-08


Helper functions

In [2]:
def generate_binary_numbers(n_bits):
    return np.array([''.join(bits) for bits in itertools.product('01', repeat=n_bits)])

def bin_to_int(bin_str):
    int_val = 0
    for i, bit in enumerate(bin_str[::-1]):
        int_val += int(bit)*(2**i)
    return int_val

def bin_to_frac(bin_str):
    frac_val = 0
    for i, bit in enumerate(bin_str):
        frac_val += int(bit)*(2**(-i-1))
    return frac_val

def frac_to_bin(frac, n_bits):
    bin_str = ""
    for i in range(n_bits):
        if frac >= 2**(-i-1):
            bin_str += "1"
            frac -= 2**(-i-1)
        else:
            bin_str += "0"
    return bin_str

## 1. Generate keys 

In [3]:
keys = generate_binary_numbers(N_BITS_IN)

## 2. Generate values

In [4]:
reciprocal_LUT_dict = {}
for val_bin in keys:
    val_int = bin_to_int(val_bin)
    reciprocal_val_frac = 1/(val_int+1)
    reciprocal_val_bin = frac_to_bin(reciprocal_val_frac, n_bits=N_BITS_OUT)
    reciprocal_LUT_dict[val_bin] = reciprocal_val_bin

## Write to verilog LUT module

In [5]:
rtl_dir = "04_ref_design"
template_fpath = os.path.join(rtl_dir, "udivision_LUT_template.sv")
with open(template_fpath, "r") as f:
    template = f.read()

In [6]:
module_rtl = template.replace("{N_BITS_IN}", str(N_BITS_IN)).replace("{N_BITS_OUT}", str(N_BITS_OUT))
module_rtl_top, module_rtl_bottom = module_rtl.split("// INSERT LUT HERE")
module_rtl_lines = [module_rtl_top]

for k,v in reciprocal_LUT_dict.items():
    line = f"        {N_BITS_IN}'b{k} : reciprocal_reg = {N_BITS_OUT}'b{v};\n"
    module_rtl_lines.append(line)

module_rtl_lines.append(module_rtl_bottom)
module_rtl = "".join(module_rtl_lines)

In [7]:
module_fpath = os.path.join(rtl_dir, f"udivision_LUT_{N_BITS_IN}bit_int_to_{N_BITS_OUT}bit_frac.sv")
with open(module_fpath, "w") as f:
    f.write(module_rtl)